In [57]:
import os
from pathlib import Path
import pandas as pd
import plotly.graph_objects as go
import urllib, json
import plotly
import numpy as np
from plotly.offline import iplot
#import plotly.plotly as py

In [58]:
#Read csv files
path = "C:\\Users\\harry\\Github\\CAP\\Capstone\\AIS_results"

shipStats = pd.read_csv(os.path.join(path,"shipStats"+".csv"),dtype=object)
voyageStats = pd.read_csv(os.path.join(path,"voyageStats"+".csv"),dtype=object)
norms = pd.read_csv(os.path.join(path,"norms"+".csv"),dtype=object)
megaStats=pd.read_csv(os.path.join(path,"megaStats"+".csv"),dtype=object)

shipTypes=["drifting_longlines",
"fixed_gear",
"pole_and_line",
"purse_seines",
"trawlers",
"trollers",
"unknown"
]
megaStats['shipNum']=megaStats.index
print("megaStats.shape=",megaStats.shape)
megaStats.columns


megaStats.shape= (354, 21)


Index(['Unnamed: 0', 'numVoy', 'mmsi', 'numObsInVoys', 'type', 'ave_len_voy',
       'ave_time_gap', 'ave_dist_gap', 'vi_dist_gap', 'vi_time_gap', 'numObs',
       'numShips', 'numVoyages', 'aveNumVoyages', 'numVoyages_q25',
       'vi_dist_diff_q75', 'vi_time_diff_q75', 'voy_len_q75', 'time_gap_q75',
       'dist_gap_q75', 'shipNum'],
      dtype='object')

In [59]:
#dff=pd.DataFrame
#print(dff.shape)
#del dff
dff=megaStats.loc[:,['mmsi','type','ave_len_voy','voy_len_q75','ave_time_gap','time_gap_q75','ave_dist_gap','dist_gap_q75','shipNum']]
#dff.reset_index
dff['flow']=1
dff.columns
condition=dff.type=='unknown'
dff.drop(dff[condition].index, inplace=True)


Index(['mmsi', 'type', 'ave_len_voy', 'voy_len_q75', 'ave_time_gap',
       'time_gap_q75', 'ave_dist_gap', 'dist_gap_q75', 'shipNum', 'flow'],
      dtype='object')

In [60]:
#Set each of the Risk Factors in a boolean column and text label column
dff['risk-v']=dff.ave_len_voy.astype(float)<dff.voy_len_q75.astype(float)
temp = {True:'Long Voyage', False:'Nominal Voyage'}
dff['risk-v-label'] = dff['risk-v'].map(temp)
dff['risk-v-label'].describe()

dff['risk-t']=dff.ave_time_gap.astype(float)>dff.time_gap_q75.astype(float)
temp = {True:'Long Time Gap', False:'Nominal Time Gap'}
dff['risk-t-label'] = dff['risk-t'].map(temp)
dff['risk-t-label'].describe()

dff['risk-d']=dff.ave_dist_gap.astype(float)>dff.dist_gap_q75.astype(float)
temp = {True:'Long Dist Gap', False:'Nominal Dist Gap'}
dff['risk-d-label'] = dff['risk-d'].map(temp)
dff['risk-d-label'].describe()

count                234
unique                 2
top       Nominal Voyage
freq                 200
Name: risk-v-label, dtype: object

count                  234
unique                   2
top       Nominal Time Gap
freq                   163
Name: risk-t-label, dtype: object

count                  234
unique                   2
top       Nominal Dist Gap
freq                   151
Name: risk-d-label, dtype: object

In [61]:
dff['risk-score']=dff['risk-v'].astype(int)+dff['risk-t'].astype(int)+dff['risk-d'].astype(int)

In [62]:
dff['risk-score'].describe()

count    234.000000
mean       0.803419
std        0.942459
min        0.000000
25%        0.000000
50%        0.000000
75%        2.000000
max        3.000000
Name: risk-score, dtype: float64

In [63]:
dff.dtypes
len(dff)
dff

mmsi            object
type            object
ave_len_voy     object
voy_len_q75     object
ave_time_gap    object
time_gap_q75    object
ave_dist_gap    object
dist_gap_q75    object
shipNum          int64
flow             int64
risk-v            bool
risk-v-label    object
risk-t            bool
risk-t-label    object
risk-d            bool
risk-d-label    object
risk-score       int32
dtype: object

234

,mmsi,type,ave_len_voy,voy_len_q75,ave_time_gap,time_gap_q75,ave_dist_gap,dist_gap_q75,shipNum,flow,risk-v,risk-v-label,risk-t,risk-t-label,risk-d,risk-d-label,risk-score
0,1252339803566.0,trawlers,1804041.0134228189,946266.0,1423.3353352869563,2538.6162744395833,1.6812849843357016,1.5939254923600168,0,1,False,Nominal Voyage,False,Nominal Time Gap,True,Long Dist Gap,1
4,5020143137211.0,trawlers,523926.29292929295,946266.0,11169.609084755353,2538.6162744395833,1.8597717094330803,1.5939254923600168,4,1,True,Long Voyage,True,Long Time Gap,True,Long Dist Gap,3
5,5145483253513.0,trawlers,47177768.666666664,946266.0,1506.748832847966,2538.6162744395833,0.8275559029767446,1.5939254923600168,5,1,False,Nominal Voyage,False,Nominal Time Gap,False,Nominal Dist Gap,0
6,5601266154050.0,drifting_longlines,20199295.42857143,5445760.0,1523.2092907462097,2684.373689727463,1.5263581600264295,1.428384245279476,6,1,False,Nominal Voyage,False,Nominal Time Gap,True,Long Dist Gap,1
9,7572518792420.0,fixed_gear,132367507.0,170571.5,969.0,5121.719308035714,0.0006518724276573968,1.5636171172796722,9,1,False,Nominal Voyage,False,Nominal Time Gap,False,Nominal Dist Gap,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349,277515277585463.0,trawlers,141132598.0,946266.0,NaN,2538.6162744395833,NaN,1.5939254923600168,349,1,False,Nominal Voyage,False,Nominal Time Gap,False,Nominal Dist Gap,0
350,278798244192815.0,drifting_longlines,26574367.4,5445760.0,NaN,2684.373689727463,NaN,1.428384245279476,350,1,False,Nominal Voyage,False,Nominal Time Gap,False,Nominal Dist Gap,0
351,280291299563683.0,fixed_gear,145459779.0,170571.5,NaN,5121.719308035714,NaN,1.5636171172796722,351,1,False,Nominal Voyage,False,Nominal Time Gap,False,Nominal Dist Gap,0
352,281060477973977.0,drifting_longlines,11751868.846153846,5445760.0,NaN,2684.373689727463,NaN,1.428384245279476,352,1,False,Nominal Voyage,False,Nominal Time Gap,False,Nominal Dist Gap,0


In [64]:
def genSankey(df,cat_cols=[],value_cols='',title='Sankey Diagram'):
    #from notes from GWU Data Viz class
    # maximum of 6 value cols -> 6 colors
    colorPalette = ['#4B8BBE','#306998','#FFE873','#FFD43B','#646464']
    labelList = []
    colorNumList = []
    for catCol in cat_cols:
        labelListTemp =  list(set(df[catCol].values))
        colorNumList.append(len(labelListTemp))
        labelList = labelList + labelListTemp
        
    # remove duplicates from labelList
    labelList = list(dict.fromkeys(labelList))
    
    # define colors based on number of levels
    colorList = []
    for idx, colorNum in enumerate(colorNumList):
        colorList = colorList + [colorPalette[idx]]*colorNum
        
    # transform df into a source-target pair
    for i in range(len(cat_cols)-1):
        if i==0:
            sourceTargetDf = df[[cat_cols[i],cat_cols[i+1],value_cols]]
            sourceTargetDf.columns = ['source','target','count']
        else:
            tempDf = df[[cat_cols[i],cat_cols[i+1],value_cols]]
            tempDf.columns = ['source','target','count']
            sourceTargetDf = pd.concat([sourceTargetDf,tempDf])
        sourceTargetDf = sourceTargetDf.groupby(['source','target']).agg({'count':'sum'}).reset_index()
        
    # add index for source-target pair
    sourceTargetDf['sourceID'] = sourceTargetDf['source'].apply(lambda x: labelList.index(x))
    sourceTargetDf['targetID'] = sourceTargetDf['target'].apply(lambda x: labelList.index(x))
    
    # creating the sankey diagram
    data = dict(
        type='sankey',
        node = dict(
          pad = 15,
          thickness = 20,
          line = dict(
            color = "black",
            width = 0.5
          ),
          label = labelList,
          color = colorList
        ),
        link = dict(
          source = sourceTargetDf['sourceID'],
          target = sourceTargetDf['targetID'],
          value = sourceTargetDf['count']
        )
      )
    
    layout =  dict(
        title = title,
        font = dict(
          size = 10
        )
    )
       
    fig = dict(data=[data], layout=layout)
    return fig

In [65]:
fig=genSankey(dff,cat_cols=['type','risk-v-label','risk-t-label','risk-d-label'],\
              value_cols='flow', \
              title='Risk Factors')
iplot(fig)
plotly.offline.plot(fig, validate=False)

'temp-plot.html'

In [75]:

dff


,mmsi,type,ave_len_voy,voy_len_q75,ave_time_gap,time_gap_q75,ave_dist_gap,dist_gap_q75,shipNum,flow,risk-v,risk-v-label,risk-t,risk-t-label,risk-d,risk-d-label,risk-score
0,1252339803566.0,trawlers,1804041.0134228189,946266.0,1423.3353352869563,2538.6162744395833,1.6812849843357016,1.5939254923600168,0,1,False,Nominal Voyage,False,Nominal Time Gap,True,Long Dist Gap,1
4,5020143137211.0,trawlers,523926.29292929295,946266.0,11169.609084755353,2538.6162744395833,1.8597717094330803,1.5939254923600168,4,1,True,Long Voyage,True,Long Time Gap,True,Long Dist Gap,3
5,5145483253513.0,trawlers,47177768.666666664,946266.0,1506.748832847966,2538.6162744395833,0.8275559029767446,1.5939254923600168,5,1,False,Nominal Voyage,False,Nominal Time Gap,False,Nominal Dist Gap,0
6,5601266154050.0,drifting_longlines,20199295.42857143,5445760.0,1523.2092907462097,2684.373689727463,1.5263581600264295,1.428384245279476,6,1,False,Nominal Voyage,False,Nominal Time Gap,True,Long Dist Gap,1
9,7572518792420.0,fixed_gear,132367507.0,170571.5,969.0,5121.719308035714,0.0006518724276573968,1.5636171172796722,9,1,False,Nominal Voyage,False,Nominal Time Gap,False,Nominal Dist Gap,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349,277515277585463.0,trawlers,141132598.0,946266.0,NaN,2538.6162744395833,NaN,1.5939254923600168,349,1,False,Nominal Voyage,False,Nominal Time Gap,False,Nominal Dist Gap,0
350,278798244192815.0,drifting_longlines,26574367.4,5445760.0,NaN,2684.373689727463,NaN,1.428384245279476,350,1,False,Nominal Voyage,False,Nominal Time Gap,False,Nominal Dist Gap,0
351,280291299563683.0,fixed_gear,145459779.0,170571.5,NaN,5121.719308035714,NaN,1.5636171172796722,351,1,False,Nominal Voyage,False,Nominal Time Gap,False,Nominal Dist Gap,0
352,281060477973977.0,drifting_longlines,11751868.846153846,5445760.0,NaN,2684.373689727463,NaN,1.428384245279476,352,1,False,Nominal Voyage,False,Nominal Time Gap,False,Nominal Dist Gap,0


count    354.000000
mean       1.531073
std        0.855176
min        0.000000
25%        1.000000
50%        2.000000
75%        2.000000
max        3.000000
Name: riskSum, dtype: float64

In [86]:
condition=dff['risk-score']==3
riskyShips=dff.loc[condition,['mmsi','type','risk-score']]

riskyShips


,mmsi,type,risk-score
4,5020143137211.0,trawlers,3
33,23593130178765.0,drifting_longlines,3
81,63716389910332.0,purse_seines,3
94,77261928739173.0,trawlers,3
120,89110306461984.0,drifting_longlines,3
135,102111574707951.0,drifting_longlines,3
153,110014060911622.0,trawlers,3
171,125954407820672.0,trollers,3
271,207595838999742.0,drifting_longlines,3
284,218997104404214.0,trawlers,3


,risk-score
4,3
33,3
81,3
94,3
120,3
135,3
153,3
171,3
271,3
284,3


In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = shipTypes,
      color = "blue"
    ),
    link = dict(
      source = [0, 1, 0, 2, 3, 3], # indices correspond to labels, eg A1, A2, A2, B1, ...
      target = [2, 3, 3, 4, 4, 5],
      value = [8, 4, 2, 8, 4, 2]
  ))])

fig.update_layout(title_text="Basic Sankey Diagram", font_size=10)
fig.show()

In [ ]:
import plotly.graph_objects as go
import urllib, json
import pandas as pd

#url = 'https://raw.githubusercontent.com/plotly/plotly.js/master/test/image/mocks/sankey_energy.json'
#response = urllib.request.urlopen(url)
#data = json.loads(response.read())
flows = pd.read_json('csvjson.json')
fig = go.Figure(data=[go.Sankey(
    valueformat = ".0f",
    valuesuffix = "TWh",
    node = dict(
      pad = 15,
      thickness = 15,
      line = dict(color = "black", width = 0.5),
      label =  data['data'][0]['node']['label'],
      color =  data['data'][0]['node']['color']
    ),
    link = dict(
      source =  data['data'][0]['link']['source'],
      target =  data['data'][0]['link']['target'],
      value =  data['data'][0]['link']['value'],
      label =  data['data'][0]['link']['label']
  ))])

fig.update_layout(
    hovermode = 'x',
    title="Energy forecast for 2050<br>Source: Department of Energy & Climate Change, Tom Counsell via <a href='https://bost.ocks.org/mike/sankey/'>Mike Bostock</a>",
    font=dict(size = 10, color = 'white'),
    plot_bgcolor='black',
    paper_bgcolor='black'
)

fig.show()